In [ ]:
# utils.py
import logging
from dotenv import load_dotenv
import os

# Configure logging
logging.basicConfig(
    filename="error.log",  # Specify your log file name
    level=logging.ERROR,   # Set the logging level
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log_error(feed_url, message):
    """
    Log an error message to the log file.

    Args:
        feed_url (str): The URL of the RSS feed where the error occurred.
        message (str): The error message to log.
    """
    logging.error(f"Feed: {feed_url} - {message}")


def get_proxy_settings(env_path="/home/kurlaj/rss_collector_project/.env"):
    """
    Load proxy settings from the .env file.

    Args:
        env_path (str): Path to the .env file.

    Returns:
        dict: A dictionary containing the proxy settings (app_proxy, username,
        password).

    Raises:
        ValueError: If required environment variables are missing.

    Example:
        >>> proxy_settings = get_proxy_settings()
        >>> proxy_settings["app_proxy"]
        "http://proxy.example.com:85"
    """
    load_dotenv(dotenv_path=env_path)

    proxy_settings = {
        "app_proxy": os.getenv("app_proxy"),
        "username": os.getenv("username"),
        "password": os.getenv("password"),
    }

    # Check for missing values
    missing_keys = [key for key, value in proxy_settings.items() if not value]
    if missing_keys:
        raise ValueError(
            f"Missing required environment variables: {', '.join(missing_keys)}"
            )

    return proxy_settings

In [ ]:
# proxy.py
from utils import get_proxy_settings
import subprocess


def fetch_feed_content_with_proxy(feed_url):
    """
    Fetch the content of an RSS feed using a proxy.

    Args:
        feed_url (str): The URL of the RSS feed.

    Returns:
        str: The raw RSS feed content as a string if successful.
        None: If fetching fails or returns a non-zero exit status.
    """
    try:
        # Retrieve proxy settings
        proxy_settings = get_proxy_settings()
        app_proxy = proxy_settings["app_proxy"]
        username = proxy_settings["username"]
        password = proxy_settings["password"]

        # Construct the curl command
        command = (
            f'echo "{password}" | '
            f'curl -s -x "{app_proxy}" -U "{username}" "{feed_url}" --proxy-ntlm'
        )

        # Execute the command
        response = subprocess.check_output(command, shell=True, text=True)
        return response

    except subprocess.CalledProcessError as e:
        print(f"Error fetching feed: {feed_url}. Exception: {e}")
        return None

    except ValueError as e:
        # This will handle missing environment variables from get_proxy_settings
        print(f"Configuration Error: {e}")
        return None


In [ ]:
# test_proxy.py

import pytest
import subprocess
from unittest.mock import patch
from rss_collector.utils import get_proxy_settings
from rss_collector.proxy import fetch_feed_content_with_proxy

def test_fetch_feed_content_with_proxy(mocker):
    """
    Test fetch_feed_content_with_proxy with detailed validation for subprocess
    command.
    """
    # Mocked proxy settings
    mock_proxy_settings = {
        "app_proxy": "http://mock_proxy.com:85",
        "username": "mock_user",
        "password": "mock_password",
    }

    # Patch get_proxy_settings to return mocked proxy settings
    mocker.patch(
        "rss_collector.utils.get_proxy_settings",
        return_value=mock_proxy_settings
        )

    # Mock subprocess.check_output to simulate RSS feed fetching
    mock_subprocess = mocker.patch(
        "subprocess.check_output", return_value="Mocked Response"
        )

    # Define a test RSS feed URL
    feed_url = "https://example.com/rss"

    # Call the function under test
    response = fetch_feed_content_with_proxy(feed_url)

    # Define the expected command that should be constructed
    expected_command = (
        f'echo "{mock_proxy_settings["password"]}" | '
        f'curl -s -x "{mock_proxy_settings["app_proxy"]}" '
        f'-U "{mock_proxy_settings["username"]}" "{feed_url}" --proxy-ntlm'
    )

    # Assert that subprocess was called with the expected command
    mock_subprocess.assert_called_once_with(
        expected_command, shell=True, text=True
        )

    # Assert that the response matches the mocked return value
    assert response == "Mocked Response"

    # Debugging output (can be removed after testing)
    print("--- Debugging Mock Application ---")
    print("Mocked proxy settings used:", mock_proxy_settings)
    print("--- Debugging Subprocess Call ---")
    print("Expected command:", expected_command)
    print("Actual call arguments:", mock_subprocess.call_args_list)
